In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from utils.data_preprocessing import load_data, preprocess_data
import os
import numpy as np

save_data = "save_data_2018/"
if not os.path.exists(save_data):
    os.mkdir(save_data)

cicids2018_path_v1 = 'data/Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv'
cicids2018_path_v2 = 'data/Friday-23-02-2018_TrafficForML_CICFlowMeter.csv'
merge_data = object
if(not os.path.isfile(save_data + "merge.csv")):
    # Optionally, remove duplicate rows
    data_2018_v1 = load_data(cicids2018_path_v1)
    data_2018_v2 = load_data(cicids2018_path_v2)

    print("Headers match. Proceeding with merge.")
    merged_df = pd.concat([data_2018_v1, data_2018_v2], axis=0, ignore_index=True)
    merged_df = merged_df.drop_duplicates()
    merged_df.to_csv(save_data + "merge.csv", index=False)
    
    merge_data = load_data(save_data + "merge.csv")
    
if(not os.path.isfile(save_data + "web_attacks.csv")):
    preprocess_data(merge_data, save_data)

Headers match. Proceeding with merge.
['Benign' 'Brute Force -Web' 'Brute Force -XSS' 'SQL Injection']
Label
Benign              2090330
Brute Force -Web        611
Brute Force -XSS        230
SQL Injection            87
Name: count, dtype: int64
ToTal begnign: 2090330
ToTal attack: 928


In [2]:
df = pd.read_csv(save_data + 'web_attacks.csv')
df['Label'] = df['Label'].apply(lambda x: 0 if x == 'Benign' else 1)


normal_df = df[df['Label'] == 0]
attack_df = df[df['Label'] != 0]

num_attack = len(attack_df)

num_normal = 2 * num_attack

In [3]:
if num_normal > len(normal_df):
    raise ValueError("num_normal exceeds the number of rows in normal_df.")
normal_df = normal_df.sample(n=num_normal, random_state=42)

# Ensure 'Label' column exists
if 'Label' not in normal_df.columns or 'Label' not in attack_df.columns:
    raise ValueError("Both DataFrames must have a 'Label' column.")

# Combine and shuffle the dataset
balanced_df = pd.concat([normal_df, attack_df]).sample(frac=1, random_state=42).reset_index(drop=True)

# Split the balanced dataset
if 'Label' not in balanced_df.columns:
    raise ValueError("The 'Label' column is missing from the balanced DataFrame.")


train_df, temp_df = train_test_split(balanced_df, test_size=0.4, random_state=42, stratify=balanced_df['Label'])
val_df, test_df = train_test_split(temp_df, test_size=0.7, random_state=42, stratify=temp_df['Label'])

# Ensure enough data for balancing
train_normal_df = train_df[train_df['Label'] == 0]
train_attack_df = train_df[train_df['Label'] == 1]

if len(train_attack_df) == 0:
    raise ValueError("There are no attack instances in the training set.")

num_train_attack = len(train_attack_df)
num_train_normal = round(2.5 * num_train_attack)

# Ensure we don't try to sample more rows than are available
if len(train_normal_df) < num_train_normal:
    num_train_normal = len(train_normal_df)  # Adjust to available number of rows

train_normal_df = train_normal_df.sample(n=num_train_normal, random_state=42)
train_df = pd.concat([train_normal_df, train_attack_df]).sample(frac=1, random_state=42).reset_index(drop=True)

In [4]:
# Print the count of each class in the training, validation, and testing sets
for name, df in zip(["Training", "Validation", "Testing"], [train_df, val_df, test_df]):
    unique, counts = np.unique(df['Label'], return_counts=True)
    print(f"{name} set class distribution:", dict(zip(unique, counts)))

Training set class distribution: {0: 1113, 1: 557}
Validation set class distribution: {0: 223, 1: 111}
Testing set class distribution: {0: 520, 1: 260}


In [5]:
# Save the train, validation, and test sets to CSV files
train_df.to_csv(save_data +  'train_set.csv', index=False)
val_df.to_csv(save_data +  'val_set.csv', index=False)
test_df.to_csv(save_data + 'test_set.csv', index=False)

In [6]:
# Define feature columns (excluding the label column)
feature_cols = [col for col in df.columns if col != 'Label']

# Extract features and labels for training, validation, and testing sets
X_train = train_df[feature_cols]
y_train = train_df['Label']
X_val = val_df[feature_cols]
y_val = val_df['Label']
X_test = test_df[feature_cols]
y_test = test_df['Label']

In [7]:
# Define classifiers
from models.decision_tree import train_decision_tree
from models.random_forest import train_random_forest
from models.lda import train_lda
from models.naive_bayes import train_naive_bayes
from models.logistic_regression import train_logistic_regression
from models.knn import train_knn
from models.svm import train_svm
from models.extra_trees import train_extra_trees
from models.bagging import train_bagging
from models.mlp import train_mlp

classifiers = {
    'Decision Tree': train_decision_tree,
    'Random Forest': train_random_forest,
    'Linear Discriminant Analysis': train_lda,
    'Naive Bayes': train_naive_bayes,
    'Logistic Regression': train_logistic_regression,
    'K-Nearest Neighbors': train_knn,
    'Support Vector Machine': train_svm,
    'Extra Trees Classifier': train_extra_trees,
    'Bagging Classifier': train_bagging,
    'Multi-layer Perceptron': train_mlp
}

In [10]:
def model_10Classifier():
    results = []
    for name, train_function in classifiers.items():
        model, val_acc, val_pre, val_rec, val_f1, test_acc, test_pre, test_rec, test_f1, time = train_function(X_train, y_train, X_val, y_val, X_test, y_test, save_data)
        results.append({
            'Model': name,
            'Validation Accuracy': val_acc,
            'Validation Precision': val_pre,
            'Validation Recall': val_rec,
            'Validation F1 Score': val_f1,
            "Test Accuracy": test_acc,
            "Test Precision": test_pre,
            "Test Recall": test_rec,
            "Test F1 Score": test_f1,
            'Time': time
        })

    # Save results to CSV
    results_df = pd.DataFrame(results)
    results_df.to_csv(save_data + '10Classsifer_results_2017.csv', index=False)
    
model_10Classifier()

Decision Tree Validation - Accuracy: 0.9191616766467066 Precision: 0.8962264150943396 Recall: 0.8558558558558559 F1 Score: 0.8755760368663594
Decision Tree Test - Accuracy: 0.926923076923077 Precision: 0.8888888888888888 Recall: 0.8923076923076924 F1 Score: 0.8905950095969289
Testing time: 0.0050 seconds
Random Forest Validation - Accuracy: 0.9341317365269461 Precision: 0.9238095238095239 Recall: 0.8738738738738738 F1 Score: 0.8981481481481481
Random Forest Test - Accuracy: 0.9448717948717948 Precision: 0.9221789883268483 Recall: 0.9115384615384615 F1 Score: 0.9168278529980658
Testing time: 0.0087 seconds
LDA Validation - Accuracy: 0.9161676646706587 Precision: 0.8217054263565892 Recall: 0.954954954954955 F1 Score: 0.8833333333333333
LDA Test - Accuracy: 0.9256410256410257 Precision: 0.8322368421052632 Recall: 0.9730769230769231 F1 Score: 0.8971631205673759
Testing time: 0.0052 seconds
Navi_bayes Validation - Accuracy: 0.6976047904191617 Precision: 0.5242718446601942 Recall: 0.97297297

c:\Users\a\anaconda3\envs\AI\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic_regression Validation - Accuracy: 0.7844311377245509 Precision: 0.7532467532467533 Recall: 0.5225225225225225 F1 Score: 0.6170212765957447
Logistic_regression Test - Accuracy: 0.7576923076923077 Precision: 0.6939890710382514 Recall: 0.48846153846153845 F1 Score: 0.5733634311512416
Testing time: 0.0111 seconds
KNN Validation - Accuracy: 0.9341317365269461 Precision: 0.9238095238095239 Recall: 0.8738738738738738 F1 Score: 0.8981481481481481
KNN Test - Accuracy: 0.9243589743589744 Precision: 0.8910505836575876 Recall: 0.8807692307692307 F1 Score: 0.8858800773694391
Testing time: 0.0523 seconds
SVM Validation - Accuracy: 0.7934131736526946 Precision: 0.9772727272727273 Recall: 0.38738738738738737 F1 Score: 0.5548387096774193
SVM Test - Accuracy: 0.7897435897435897 Precision: 1.0 Recall: 0.36923076923076925 F1 Score: 0.5393258426966292
Testing time: 0.0435 seconds
Extra_trees Validation - Accuracy: 0.9341317365269461 Precision: 0.9238095238095239 Recall: 0.8738738738738738 F1 Score

In [11]:
import pandas as pd
from sklearn.decomposition import PCA

# Assume the classifiers dictionary and train functions are already defined
# Define a function to apply PCA and train classifiers

def train_with_pca(X_train, y_train, X_val, y_val, X_test, y_test, n_components=20):
    X_combined = np.concatenate((X_train, X_val, X_test), axis=0)
    y_combined = np.concatenate((y_train, y_val, y_test), axis=0)

    pca = PCA(n_components=n_components)
    X_combined_pca = pca.fit_transform(X_combined)

    # Bước 3: Tách lại dữ liệu thành các tập train, validation và test
    X_train_pca = X_combined_pca[:len(X_train)]
    X_val_pca = X_combined_pca[len(X_train):len(X_train)+len(X_val)]
    X_test_pca = X_combined_pca[len(X_train)+len(X_val):]

    # Kiểm tra kích thước của các tập dữ liệu sau khi PCA
    print("X_train_pca shape:", X_train_pca.shape)
    print("X_val_pca shape:", X_val_pca.shape)
    print("X_test_pca shape:", X_test_pca.shape)
    
    results = []
    for name, train_function in classifiers.items():
        model, val_acc, val_pre, val_rec, val_f1, test_acc, test_pre, test_rec, test_f1, time = train_function(X_train_pca, y_train, X_val_pca, y_val, X_test_pca, y_test, save_data)
        results.append({
            'Model': name,
            'Validation Accuracy': val_acc,
            'Validation Precision': val_pre,
            'Validation Recall': val_rec,
            'Validation F1 Score': val_f1,
            'Test Accuracy': test_acc,
            'Test Precision': test_pre,
            'Test Recall': test_rec,
            'Test F1 Score': test_f1,
            'Time': time
        })
    
    return results

# Call the function and save results
pca_results = train_with_pca(X_train, y_train, X_val, y_val, X_test, y_test)
pca_results_df = pd.DataFrame(pca_results)
pca_results_df.to_csv(save_data + 'pca_model_results_2018.csv', index=False)


X_train_pca shape: (1670, 20)
X_val_pca shape: (334, 20)
X_test_pca shape: (780, 20)
Decision Tree Validation - Accuracy: 0.9161676646706587 Precision: 0.9191919191919192 Recall: 0.8198198198198198 F1 Score: 0.8666666666666667
Decision Tree Test - Accuracy: 0.9256410256410257 Precision: 0.8976377952755905 Recall: 0.8769230769230769 F1 Score: 0.8871595330739299
Testing time: 0.0039 seconds
Random Forest Validation - Accuracy: 0.9221556886227545 Precision: 0.912621359223301 Recall: 0.8468468468468469 F1 Score: 0.8785046728971962
Random Forest Test - Accuracy: 0.9358974358974359 Precision: 0.9038461538461539 Recall: 0.9038461538461539 F1 Score: 0.9038461538461539
Testing time: 0.0125 seconds
LDA Validation - Accuracy: 0.7844311377245509 Precision: 0.8823529411764706 Recall: 0.40540540540540543 F1 Score: 0.5555555555555556
LDA Test - Accuracy: 0.7769230769230769 Precision: 0.9056603773584906 Recall: 0.36923076923076925 F1 Score: 0.5245901639344263
Testing time: 0.0076 seconds
Navi_bayes Va

c:\Users\a\anaconda3\envs\AI\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic_regression Validation - Accuracy: 0.8053892215568862 Precision: 0.8709677419354839 Recall: 0.4864864864864865 F1 Score: 0.6242774566473989
Logistic_regression Test - Accuracy: 0.791025641025641 Precision: 0.8211920529801324 Recall: 0.47692307692307695 F1 Score: 0.6034063260340633
Testing time: 0.0031 seconds
KNN Validation - Accuracy: 0.9311377245508982 Precision: 0.9313725490196079 Recall: 0.8558558558558559 F1 Score: 0.892018779342723
KNN Test - Accuracy: 0.9205128205128205 Precision: 0.889763779527559 Recall: 0.8692307692307693 F1 Score: 0.8793774319066148
Testing time: 0.0299 seconds
SVM Validation - Accuracy: 0.7934131736526946 Precision: 0.9772727272727273 Recall: 0.38738738738738737 F1 Score: 0.5548387096774193
SVM Test - Accuracy: 0.7897435897435897 Precision: 1.0 Recall: 0.36923076923076925 F1 Score: 0.5393258426966292
Testing time: 0.0442 seconds
Extra_trees Validation - Accuracy: 0.9191616766467066 Precision: 0.9117647058823529 Recall: 0.8378378378378378 F1 Score: 0

In [12]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Định nghĩa Autoencoder
# Định nghĩa Autoencoder
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        encoding_dim1 = 52
        encoding_dim2 = 26
        encoding_dim3 = 10

        # Lớp mã hóa
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoding_dim1),
            nn.BatchNorm1d(encoding_dim1),  # Thêm BatchNorm1d
            nn.ReLU(),
            nn.Linear(encoding_dim1, encoding_dim2),
            nn.BatchNorm1d(encoding_dim2),  # Thêm BatchNorm1d
            nn.ReLU(),
            nn.Linear(encoding_dim2, encoding_dim3),
            nn.BatchNorm1d(encoding_dim3),  # Thêm BatchNorm1d
            nn.ReLU()
        )
        # Lớp giải mã
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim3, encoding_dim2),
            nn.BatchNorm1d(encoding_dim2),  # Thêm BatchNorm1d
            nn.ReLU(),
            nn.Linear(encoding_dim2, encoding_dim1),
            nn.BatchNorm1d(encoding_dim1),  # Thêm BatchNorm1d
            nn.ReLU(),
            nn.Linear(encoding_dim1, input_dim),
            nn.Sigmoid()  # Giữ sigmoid cho đầu ra
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded, encoded

# Hàm để áp dụng deep autoencoder và train classifiers
def train_with_autoencoder(X_train, y_train, X_val, y_val, X_test, y_test, epochs=10, batch_size=256, alpha1=0.1):
    input_dim = X_train.shape[1]
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Khởi tạo mô hình và các thành phần cần thiết
    autoencoder = Autoencoder(input_dim).to(device)
    optimizer = optim.AdamW(autoencoder.parameters(), lr=0.0001)
    criterion = nn.MSELoss()

    X_train_tensor = torch.tensor(X_train.to_numpy(), dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val.to_numpy(), dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val.to_numpy(), dtype=torch.float32)

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Training loop
    for epoch in range(epochs):
        autoencoder.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            decode, encode = autoencoder(inputs)
            loss = alpha1 * criterion(decode, inputs)  # Chỉ sử dụng MSE loss
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(autoencoder.parameters(), max_norm=1.0)  # Gradient Clipping
            optimizer.step()
            train_loss += loss.item()

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {train_loss/len(train_loader)}')

    autoencoder.eval()
    with torch.no_grad():
        X_train_enc = autoencoder.encoder(X_train_tensor.to(device)).cpu().numpy()
        X_val_enc = autoencoder.encoder(X_val_tensor.to(device)).cpu().numpy()
        X_test_enc = autoencoder.encoder(torch.tensor(X_test.to_numpy(), dtype=torch.float32).to(device)).cpu().numpy()
    
    # Train và đánh giá classifiers
    results = []
    for name, train_function in classifiers.items():
        model, val_acc, val_pre, val_rec, val_f1, test_acc, test_pre, test_rec, test_f1, time = train_function(X_train_enc, y_train, X_val_enc, y_val, X_test_enc, y_test, save_data)
        results.append({
            'Model': name,
            'Validation Accuracy': val_acc,
            'Validation Precision': val_pre,
            'Validation Recall': val_rec,
            'Validation F1 Score': val_f1,
            'Test Accuracy': test_acc,
            'Test Precision': test_pre,
            'Test Recall': test_rec,
            'Test F1 Score': test_f1,
            'Time': time
        })
    
    return results

# Gọi hàm và lưu kết quả
autoencoder_results = train_with_autoencoder(X_train, y_train, X_val, y_val, X_test, y_test)
autoencoder_results_df = pd.DataFrame(autoencoder_results)
autoencoder_results_df.to_csv(save_data + 'autoencoder_model_results.csv', index=False)

Epoch [1/10], Loss: 11083099593581.715
Epoch [2/10], Loss: 11106696298496.0
Epoch [3/10], Loss: 11252736121709.715
Epoch [4/10], Loss: 10927725197019.428
Epoch [5/10], Loss: 10882078137197.715
Epoch [6/10], Loss: 10784192519021.715
Epoch [7/10], Loss: 11163980005376.0
Epoch [8/10], Loss: 10932495319040.0
Epoch [9/10], Loss: 10811294313910.857
Epoch [10/10], Loss: 10840765815661.715
Decision Tree Validation - Accuracy: 0.9041916167664671 Precision: 0.8910891089108911 Recall: 0.8108108108108109 F1 Score: 0.8490566037735849
Decision Tree Test - Accuracy: 0.9128205128205128 Precision: 0.8636363636363636 Recall: 0.8769230769230769 F1 Score: 0.8702290076335878
Testing time: 0.0000 seconds
Random Forest Validation - Accuracy: 0.9191616766467066 Precision: 0.9117647058823529 Recall: 0.8378378378378378 F1 Score: 0.8732394366197183
Random Forest Test - Accuracy: 0.9294871794871795 Precision: 0.8957528957528957 Recall: 0.8923076923076924 F1 Score: 0.8940269749518305
Testing time: 0.0106 seconds
L

In [13]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import random
from sklearn import preprocessing  # Import the preprocessing module from sklearn


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Cố định seed
set_seed(42)

# Định nghĩa Autoencoder
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        encoding_dim1 = 40
        encoding_dim2 = 20
        encoding_dim3 = 10
        encoding_dim4 = 20
        encoding_dim5 = 12

        # Lớp mã hóa
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoding_dim1),
            nn.BatchNorm1d(encoding_dim1),  # Thêm BatchNorm1d
            nn.Tanh(),
            nn.Linear(encoding_dim1, encoding_dim2),
            nn.BatchNorm1d(encoding_dim2),  # Thêm BatchNorm1d
            nn.Tanh(),
            nn.Linear(encoding_dim2, encoding_dim3),
            nn.BatchNorm1d(encoding_dim3),  # Thêm BatchNorm1d
            # nn.Linear(encoding_dim3, encoding_dim4),
            # nn.BatchNorm1d(encoding_dim4),  # Thêm BatchNorm1d
            # nn.Tanh(),
            # nn.Linear(encoding_dim4, encoding_dim5),
            # nn.BatchNorm1d(encoding_dim5),  # Thêm BatchNorm1d
            nn.Tanh(),
        )
        self.decoder = nn.Sequential(
            # nn.Linear(encoding_dim5, encoding_dim4),
            # nn.BatchNorm1d(encoding_dim4),  # Thêm BatchNorm1d
            # nn.Tanh(),
            # nn.Linear(encoding_dim4, encoding_dim3),
            # nn.BatchNorm1d(encoding_dim3),  # Thêm BatchNorm1d
            # nn.Tanh(),
            nn.Linear(encoding_dim3, encoding_dim2),
            nn.BatchNorm1d(encoding_dim2),  # Thêm BatchNorm1d
            nn.Tanh(),
            nn.Linear(encoding_dim2, encoding_dim1),
            nn.BatchNorm1d(encoding_dim1),  # Thêm BatchNorm1d
            nn.Tanh(),
            nn.Linear(encoding_dim1, input_dim), 
            nn.Tanh(),
        )
        self._initialize_weights()
    
    def _initialize_weights(self):
        for layer in self.encoder:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                nn.init.zeros_(layer.bias)
        for layer in self.decoder:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                nn.init.zeros_(layer.bias)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded, encoded

# Định nghĩa BinaryMagnetLoss
class BinaryMagnetLoss(nn.Module):
    def __init__(self, D=10, M=2, alpha=7.18, epsilon=0):
        super(BinaryMagnetLoss, self).__init__()
        self.D = D
        self.M = M
        self.alpha = alpha
        self.epsilon = epsilon

    def forward(self, outputs, indices, assignment):
        outputs = outputs.float()
        unique_clusters = torch.unique(assignment[indices])
        batch_clusters = {cluster.item(): (assignment[indices] == cluster).nonzero(as_tuple=True)[0] for cluster in unique_clusters}

        num_instances = sum(len(batch_clusters[cluster.item()]) for cluster in unique_clusters)
        c_means = torch.stack([outputs[batch_clusters[cluster.item()]].mean(dim=0) for cluster in unique_clusters])

        stdev = sum((outputs[i] - c_means[unique_clusters.tolist().index(assignment[indices[i]])]).norm(p=2).pow(2) 
                    for i in range(len(indices))) / max(num_instances - 1, 1)
        stdev = -2.0 * (stdev + self.epsilon)

        if torch.isnan(stdev).any():
            stdev = torch.tensor(1.0, device=outputs.device)  # Gán giá trị hợp lý nếu phát hiện NaN

        loss = torch.zeros(1).to(outputs.device)
        denom = [torch.zeros(1).to(outputs.device) for _ in range(len(indices))]

        for m, cluster in enumerate(unique_clusters):
            cluster_indices = batch_clusters[cluster.item()]
            for d in range(min(self.D, len(cluster_indices))):
                ind = cluster_indices[d].item()
                for other_cluster in unique_clusters:
                    if other_cluster != cluster:
                        denom[ind] += ((outputs[ind] - c_means[unique_clusters.tolist().index(other_cluster)]).norm().pow(2) / stdev).exp()

                loss_term = (((outputs[ind] - c_means[m]).norm().pow(2) / stdev - self.alpha).exp() / (denom[ind] + self.epsilon)).log().clamp(max=0.0)
                if torch.isnan(loss_term).any() or torch.isinf(loss_term).any():
                    continue
                loss -= loss_term

        loss /= num_instances
        return loss

# Custom Loss Function combining Cross-Entropy and Binary Magnet Loss
def custom_loss(y_true, y_pred, embeddings, inputs, alpha1=1, alpha2=1):
    mse_loss = F.mse_loss(inputs, y_pred) 
    magnet_loss_fn = BinaryMagnetLoss()
    magnet_loss_value = magnet_loss_fn(embeddings, torch.arange(len(y_true)), y_true)
    return alpha1 * magnet_loss_value + alpha2 * mse_loss

# Hàm để áp dụng deep autoencoder và train classifiers
def train_with_autoencoder(X_train, y_train, X_val, y_val, X_test, y_test, classifiers, save_data, epochs, batch_size=256, alpha1=1, alpha2=1):
    input_dim = X_train.shape[1]
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    autoencoder = Autoencoder(input_dim).to(device)
    optimizer = optim.Adadelta(autoencoder.parameters(), lr=1)

    
    scaler = preprocessing.MaxAbsScaler()
    scaler.fit(X_train)  # Fit the scaler on the training data
    
    X_train = scaler.transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)
    
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val.to_numpy(), dtype=torch.float32)

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    for epoch in range(epochs):
        autoencoder.train()
        train_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            decoded, encoded = autoencoder(inputs)
            loss = custom_loss(labels, decoded, encoded, inputs, alpha1=alpha1, alpha2=alpha2)
            if torch.isnan(loss).any() or torch.isinf(loss).any():
                print(f'Invalid loss detected at epoch {epoch+1}. Skipping this batch.')
                continue
            loss.backward()
            #torch.nn.utils.clip_grad_norm_(autoencoder.parameters(), max_norm=1.0)  # Gradient Clipping
            optimizer.step()
            train_loss += loss.item()

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {train_loss/len(train_loader)}')

    autoencoder.eval()
    with torch.no_grad():
        X_train_enc = autoencoder.encoder(X_train_tensor.to(device)).cpu().numpy()
        X_val_enc = autoencoder.encoder(X_val_tensor.to(device)).cpu().numpy()
        X_test_enc = autoencoder.encoder(torch.tensor(X_test, dtype=torch.float32).to(device)).cpu().numpy()

    results = []
    try:
        for name, train_function in classifiers.items():
            model, val_acc, val_pre, val_rec, val_f1, test_acc, test_pre, test_rec, test_f1, time = train_function(X_train_enc, y_train, X_val_enc, y_val, X_test_enc, y_test, save_data)
            #if(test_acc >= 0.979):
            results.append({
                'Model': name,
                'Validation Accuracy': val_acc,
                'Validation Precision': val_pre,
                'Validation Recall': val_rec,
                'Validation F1 Score': val_f1,
                'Test Accuracy': test_acc,
                'Test Precision': test_pre,
                'Test Recall': test_rec,
                'Test F1 Score': test_f1,
                'epochs': epochs,
                'alpha1': alpha1,
                'alpha2': alpha2,
                'Time': time
            })
    except Exception as e:
        print(f"Error: {e}")

    return results

def generate_random_choice():
    num1 = random.randint(0, 10)
    num2 = random.randint(0, 10)
    return num1, num2

def generate_random_choice1():
    values = [i/100 for i in range(1, 11)]
    num1 = random.choice(values)
    num2 = random.choice(values)
    return num1, num2

def generate_random_choice2():
    values = [i/10 for i in range(1, 11)]
    num1 = random.choice(values)
    num2 = random.choice(values)
    return num1, num2

def generate_random_choice3():
    values1 = [i/10 for i in range(1, 11)]
    values2 = [i/100 for i in range(1, 11)]
    num1 = random.choice(values1)
    num2 = random.choice(values2)
    return num1, num2

def generate_random_choice4():
    values1 = [i/100 for i in range(1, 11)]
    values2 = [i/10 for i in range(1, 11)]
    num1 = random.choice(values1)
    num2 = random.choice(values2)
    return num1, num2

maxx  = 50
for rd in range(1, maxx):
    if rd < maxx/5:
        alpha1, alpha2 = generate_random_choice()
    elif rd < 2*maxx/5:
        alpha1, alpha2 = generate_random_choice1()
    if rd < 3*maxx/5:
        alpha1, alpha2 = generate_random_choice2()
    elif rd < 4*maxx/5:
        alpha1, alpha2 = generate_random_choice3()
    else:
        alpha1, alpha2 = generate_random_choice4()
# alpha1 = 1
# alpha2 = 0.1

    autoencoder_results = train_with_autoencoder(X_train, y_train, X_val, y_val, X_test, y_test, classifiers, save_data, epochs=6, alpha1=alpha1, alpha2=alpha2)
    autoencoder_results_df = pd.DataFrame(autoencoder_results)

    csv_file = save_data + 'autoencoder_Binary_Magnet_Loss_results_2018_20_8_new_3lop.csv'

    with open(csv_file, mode='a+') as file:
        file.write('\n')  # Thêm một dòng trống
    autoencoder_results_df.to_csv(csv_file, mode='a+', header=True, index=False)

Epoch [1/6], Loss: 0.20811895174639566
Epoch [2/6], Loss: 0.15550751558371953
Epoch [3/6], Loss: 0.13587791046925954
Epoch [4/6], Loss: 0.12465270502226693
Epoch [5/6], Loss: 0.11968271008559636
Epoch [6/6], Loss: 0.11341399593012673
Decision Tree Validation - Accuracy: 0.9251497005988024 Precision: 0.8981481481481481 Recall: 0.8738738738738738 F1 Score: 0.8858447488584474
Decision Tree Test - Accuracy: 0.9435897435897436 Precision: 0.9153846153846154 Recall: 0.9153846153846154 F1 Score: 0.9153846153846154
Testing time: 0.0025 seconds
Random Forest Validation - Accuracy: 0.9341317365269461 Precision: 0.908256880733945 Recall: 0.8918918918918919 F1 Score: 0.9
Random Forest Test - Accuracy: 0.9512820512820512 Precision: 0.9236641221374046 Recall: 0.9307692307692308 F1 Score: 0.9272030651340997
Testing time: 0.0106 seconds
LDA Validation - Accuracy: 0.8922155688622755 Precision: 0.8099173553719008 Recall: 0.8828828828828829 F1 Score: 0.8448275862068966
LDA Test - Accuracy: 0.8948717948717